In [1]:
import PySimpleGUI as sg


layout = [[sg.Text('１行目です')],
          [sg.Text('２行目です')],
          [sg.Text('３行目です'), sg.Text('これも３行目です')]]

window = sg.Window('sample', layout)

event, values = window.read()
window.close()

# chatgptパイセンのコード

In [2]:
# import tkinter as tk
# from tkinter import filedialog
# import pandas as pd
# from gensim.models import Word2Vec
# from sklearn.metrics.pairwise import cosine_similarity

# class App:
#     def __init__(self, master):
#         self.master = master
#         master.title("Similarity Calculator")

#         self.label = tk.Label(master, text="Enter CSV path:")
#         self.label.pack()

#         self.path_entry = tk.Entry(master)
#         self.path_entry.pack()

#         self.browse_button = tk.Button(master, text="Browse", command=self.browse)
#         self.browse_button.pack()

#         self.load_button = tk.Button(master, text="Load", command=self.load_data)
#         self.load_button.pack()

#         self.label = tk.Label(master, text="Enter a sentence:")
#         self.label.pack()

#         self.sentence_entry = tk.Entry(master)
#         self.sentence_entry.pack()

#         self.calculate_button = tk.Button(master, text="Calculate Similarity", command=self.calculate)
#         self.calculate_button.pack()

#         self.result_label = tk.Label(master, text="")
#         self.result_label.pack()

#     def browse(self):
#         filepath = filedialog.askopenfilename()
#         self.path_entry.delete(0, tk.END)
#         self.path_entry.insert(0, filepath)

#     def load_data(self):
#         filepath = self.path_entry.get()
#         self.df = pd.read_csv(filepath)

#     def calculate(self):
#         sentence = self.sentence_entry.get()
#         model = Word2Vec(self.df['text'].str.split())
#         sentence_vector = model.wv[sentence.split()]
#         similarity = cosine_similarity(sentence_vector.reshape(1, -1), model.wv.vectors)
#         top10 = similarity.argsort()[0][-11:-1][::-1]
#         result_string = '\n'.join(self.df.iloc[top10]['text'])
#         self.result_label.config(text=result_string)

# root = tk.Tk()
# app = App(root)
# root.mainloop()


In [3]:
# import tkinter as tk
# from tkinter import filedialog
# import pandas as pd
# from gensim.models import Word2Vec
# from sklearn.metrics.pairwise import cosine_similarity

# class App:
#     def __init__(self, master):
#         self.master = master
#         master.title("Similarity Calculator")

#         self.label = tk.Label(master, text="Enter CSV path:")
#         self.label.pack()

#         self.path_entry = tk.Entry(master)
#         self.path_entry.pack()

#         self.browse_button = tk.Button(master, text="Browse", command=self.browse)
#         self.browse_button.pack()

#         self.load_button = tk.Button(master, text="Load", command=self.load_data)
#         self.load_button.pack()

#         self.label = tk.Label(master, text="Enter a sentence:")
#         self.label.pack()

#         self.sentence_entry = tk.Entry(master)
#         self.sentence_entry.pack()

#         self.calculate_button = tk.Button(master, text="Calculate Similarity", command=self.calculate)
#         self.calculate_button.pack()

#         self.result_label = tk.Label(master, text="")
#         self.result_label.pack()

#     def browse(self):
#         filepath = filedialog.askopenfilename()
#         self.path_entry.delete(0, tk.END)
#         self.path_entry.insert(0, filepath)

#     def load_data(self):
#         filepath = self.path_entry.get()
#         self.df = pd.read_csv(filepath)

#     def calculate(self):
#         sentence = self.sentence_entry.get()
#         model = Word2Vec(self.df['要約'].str.split())
#         sentence_vector = model.wv[sentence.split()]
#         similarity = cosine_similarity(sentence_vector.reshape(1, -1), model.wv.vectors)
#         top10 = similarity.argsort()[0][-11:-1][::-1]
#         result_string = '\n'.join(self.df.iloc[top10]['要約'])
#         self.result_label.config(text=result_string)

# root = tk.Tk()
# app = App(root)
# root.mainloop()


In [1]:
import pickle

def pickle_dump(obj, path):
    with open(path, mode='wb') as f:
        pickle.dump(obj,f)

def pickle_load(path):
    with open(path, mode='rb') as f:
        data = pickle.load(f)
        return data

In [3]:
import tkinter as tk
from tkinter import ttk
from tkinter import filedialog
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer, util

class App:
    def __init__(self, master):
        self.model = SentenceTransformer('stsb-xlm-r-multilingual') 
        self.df = pd.read_pickle('./df_input_default.pkl')
        self.document_vecs = pickle_load('./document_vecs_default.pkl')

        self.master = master
        master.title("Similarity Calculator")

        self.label = tk.Label(master, text="Enter CSV path:")
        self.label.pack()

        self.path_entry = tk.Entry(master)
        self.path_entry.pack()

        self.browse_button = tk.Button(master, text="Browse", command=self.browse)
        self.browse_button.pack()

        self.load_button = tk.Button(master, text="Load", command=self.load_data)
        self.load_button.pack()

        self.label = tk.Label(master, text="Enter a sentence:")
        self.label.pack()

        self.sentence_entry = tk.Entry(master)
        self.sentence_entry.pack()

        self.calculate_button = tk.Button(master, text="Calculate Similarity", command=self.calculate)
        self.calculate_button.pack()

        self.result_label = tk.Label(master, text="",anchor='e', justify='left')
        self.result_label.pack()

    def browse(self):
        filepath = filedialog.askopenfilename()
        self.path_entry.delete(0, tk.END)
        self.path_entry.insert(0, filepath)

    def load_data(self):
        filepath = self.path_entry.get()
        self.df = pd.read_csv(filepath).dropna(subset=["要約"], inplace=True)
        document_vecs=[]
        for doc in self.df["要約"].values:
            document_vecs.append(self.model.encode(doc))
        self.document_vecs = document_vecs

    def calculate(self):
        sentence = self.sentence_entry.get()
        input_vec = self.model.encode(sentence)
        rank_size = 10
        v1 = np.linalg.norm(input_vec)
        cos_sim = []
        for v2 in self.document_vecs:
            cos_sim.append( np.dot(input_vec,v2)/(v1*np.linalg.norm(v2)) )
            doc_sort = np.argsort(np.array(cos_sim))[::-1]
            cos_sort = sorted(cos_sim,reverse=True)
        
        result_label = ''
        for i in range(rank_size):
            print(cos_sort[i])
            print(self.df["要約"].values[doc_sort[i]])
            result_label += '\n' + str(cos_sort[i]) + ', ' + self.df["要約"].values[doc_sort[i]]
        
        self.result_label.config(text=result_label)
        self.result_label.pack()

root = tk.Tk()
app = App(root)
root.mainloop()

0.704894
(57)【要約】
【課題】乗員が見やすいように右左折や車線変更の表示<BR>を行う。
【解決手段】少なくとも車両の操舵を伴う自動運転を行<BR>う車両制御部を有する車両用の表示装置が提供される。<BR>この表示装置の表示領域は、車室内の前部座席に対向し<BR>て、運転席の前方から助手席の前方まで車幅方向に延在<BR>し、進行方向を車内に表示する進行方向表示部を含む。<BR>表示装置は、方向指示器が運転者により操作された場合<BR>には、操作に応じた進行方向を前記運転席の正面の進行<BR>方向表示部に表示し、車両制御部が主体で車両が制御さ<BR>れている場合には、進行方向を車両中央部の進行方向表<BR>示部に表示する。
【選択図】図４

0.7011814
(57)【要約】
  自動運転表示システム（３９９９）は、自動運転を実<BR>行する自動運転制御装置（３３００）と、車両（３００<BR>０）の外部から表示状態が判別可能な自動運転表示装置<BR>（３９００）と、自動運転を指示する自動運転指示信号<BR>を受け付けた場合に自動運転制御装置（３３００）に車<BR>両（３０００）の自動運転を実行させると共に、自動運<BR>転制御装置（３３００）による車両（３０００）の自動<BR>運転の開始を監視し、自動運転制御装置（３３００）に<BR>よって車両（３０００）の自動運転が開始された場合に<BR>、自動運転が開始される前と異なる表示状態に、自動運<BR>転表示装置（３９００）の表示状態を制御する車載器（<BR>３５００）とを備えた。

0.6967863
(57)【要約】      （修正有）
【課題】自動運転モードから手動運転モードに切り替わ<BR>る際に、運転者の注意を迅速に自動車両の誘導に向ける<BR>。
【解決手段】自動車両１が、運転者により誘導される手<BR>動運転モードと、制御装置３により自動的に実施される<BR>自動運転モードとの間で切り替えられ、少なくとも手動<BR>運転モードにおいて、車両固有の表示装置４が、操舵ハ<BR>ンドル７に配設された入力装置によって操作される方法<BR>において、通信接続１５が、一方では、自動車両１の通<BR>信装置１３と、自動車両１から離間している携帯通信端<BR>末１４との間に確立され、自動車両１の自動運転モー